# Capítulo 13: Cálculos químico-quânticos com a biblioteca ASE

Livro: Aprendendo Química com Python, Rodrigo Q. de Albuquerque & Gerd B. da Rocha, 2021, Amazon Book.

E-Mail: learn.chem.python@gmail.com


## Versão do Python usada

In [1]:
import sys
print(sys.version)

3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
[GCC 9.3.0]


### Caixa de sessão interativa 45: ASE: Calculando propriedades

In [2]:
from ase.calculators.mopac import MOPAC
from ase.io import read
from ase.units import Debye

In [5]:
mol = read('h2.xyz')            # distancia de ligação = 0.7A
path_mopac = '/home/gerd/progs/mopac/'  # Aqui coloque o caminho para o executável do MOPAC
comando_MOPAC = path_mopac + 'MOPAC2016.exe PREFIX.mop'

# Esse é o comando para executar o MOPAC já indicando onde o executável se encontra
print(comando_MOPAC)            

/home/gerd/progs/mopac/MOPAC2016.exe PREFIX.mop


In [6]:
mol.calc = MOPAC(command = comando_MOPAC, method='RM1')
# Para menos verbosidade (Linux/Mac): substitua 'PREFIX.mop' por 'PREFIX.mop 2>/dev/null'
# Windows: command = 'C:\Program Files\MOPAC\MOPAC2016.exe PREFIX.mop'
pe = mol.get_potential_energy()                       # em eV
homo, lumo = mol.calc.get_homo_lumo_levels()          # em eV
d = mol.get_dipole_moment() / Debye       # Momento de dipolo
print(f'A energia potencial da molécula é {pe} eV')
print(f'Energias de HOMO e LUMO = {homo} e {lumo} eV')
print(f'Momento de dipolo = {d} Debye')

A energia potencial da molécula é -28.52254 eV
Energias de HOMO e LUMO = -15.04099 e 5.10285 eV
Momento de dipolo = [0. 0. 0.] Debye


In [7]:
import numpy as np
np.linalg.norm(d)

0.0

### Caixa de sessão interativa 46: ASE: Otimizando geometrias

In [8]:
%%file hf.xyz
2

H       -0.35000000       0.00000000       0.00000000
F        0.35000000       0.00000000       0.00000000

Writing hf.xyz


In [10]:
from ase.optimize import BFGS
from ase.calculators.mopac import MOPAC
from ase.io import read, write

In [11]:
path_mopac = '/home/gerd/progs/mopac/'  # Aqui coloque o caminho para o executável do MOPAC

In [12]:
mol = read('hf.xyz')                         # molécula de HF
cmd = path_mopac + 'MOPAC2016.exe PREFIX.mop 2>/dev/null'
mol.calc = MOPAC(command = cmd, method='RM1')
optimizer = BFGS(mol)
optimizer.run()                         # otimize a estrutura

      Step     Time          Energy         fmax
BFGS:    0 10:49:26     -501.504140       13.0807
BFGS:    1 10:49:26     -502.251750        4.0006
BFGS:    2 10:49:26     -502.508730        1.8511
BFGS:    3 10:49:26     -502.573860        0.1745
BFGS:    4 10:49:26     -502.574380        0.0149


True

In [13]:
mol.get_positions()                  # coordenadas otimizadas

array([[-0.45865878,  0.        ,  0.        ],
       [ 0.45865878,  0.        ,  0.        ]])

In [14]:
mol.get_forces()

array([[ 0.01491894, -0.        , -0.        ],
       [-0.01491894, -0.        , -0.        ]])

In [15]:
write('hf_opt.xyz', mol)         # salve a molécula otimizada
calculator = mol.get_calculator()
hof = calculator.final_hof         # Calor de formação, em eV
hof                                # Calor de formação, em eV

-2.939487819826392

In [17]:
from ase.units import kcal, kJ
from ase.units import mol as mol_c

In [18]:
hof * mol_c / kcal
# 'hof', em kcal/mol

-67.7862

In [19]:
hof * mol_c / kJ
# 'hof', em kJ/mol

-283.61746079999995

### Caixa de código 35: ASE: Otimizando e visualizando várias moléculas

In [ ]:
path_mopac = '/home/gerd/progs/mopac/'  # Aqui coloque o caminho para o executável do MOPAC

In [29]:
%%file Mol1.xyz
8

N         -3.99852        0.55342       -2.09365
C         -2.01988        0.62281       -1.23425
O         -1.06219        1.71839       -1.36477
H         -4.44726       -0.32768       -1.84326
H         -3.93089        0.62404       -3.10896
H         -1.51574       -0.23738       -1.62259
H         -2.27507        0.72758       -0.20042
H         -0.73187        1.75224       -2.27617  

Writing Mol1.xyz


In [30]:
%%file Mol2.xyz
4

B       -0.00000000       0.00000000       0.00000000
F        1.10800000       0.00000000       0.00000000
F       -0.55400000      -0.91751000       0.28093000
F       -0.55400000       0.91751000      -0.28093000    

Writing Mol2.xyz


In [31]:
%%file Mol3.xyz
15

C         -4.34326        1.19612        0.00000
C         -4.23694        2.69527        0.00000
C         -2.32314        2.45073        0.00000
C         -2.29124        0.93032        0.00000
C         -3.42889       -0.15417        0.00000
H         -4.93991        1.03599       -0.87365
H         -4.43960        1.17027        1.06534
H         -4.62848        3.17311        0.87365
H         -4.73305        3.30073       -0.72951
H         -1.86376        2.86377        0.87365
H         -1.79347        2.92698       -0.79846
H         -1.72085        0.69307        0.87365
H         -2.02413        0.81922       -1.03015
H         -3.49489       -0.76840       -0.87365
H         -3.51556       -0.96078        0.69769  

Writing Mol3.xyz


In [32]:
%%file Mol4.xyz
5

C         -2.73780        1.11107        0.00000
Cl        -1.66780        1.11107        0.00000
H         -3.09446        1.87854       -0.65474
H         -3.09446        1.29435        0.99202
Cl        -3.09447        0.16031       -0.33727  

Writing Mol4.xyz


In [33]:
%%file Mol5.xyz
5

C       -0.00000000       0.00000000       0.00000000
H        1.07000000       0.00000000       0.00000000
H       -0.35666000      -0.45677000      -0.89947000
H       -0.35667000       1.00735000       0.05416000
H       -0.35667000      -0.55058000       0.84531000    

Writing Mol5.xyz


In [34]:
%%file Mol6.xyz
9

C         -5.46668        2.18202       -0.66973
C         -3.62099        2.21961       -0.35211
O         -2.74447        3.33770       -0.07126
H         -5.77849        1.17304       -0.84187
H         -5.69275        2.77688       -1.52992
H         -5.98309        2.57468        0.18118
H         -3.19727        1.76959       -1.22553
H         -3.64083        1.80659        0.63477
H         -2.60445        3.85053       -0.88261  

Writing Mol6.xyz


In [35]:
%%file Mol7.xyz
3

O        0.03991322       0.05636463       0.00303043
H       -0.31674678      -0.40040537      -0.89643957
H       -0.31675678      -0.49421537       0.84834043    

Writing Mol7.xyz


In [36]:
%%file Mol8.xyz
4

N       -0.02012330      -0.00086057       0.05691048
H        0.99987670      -0.00086057       0.05691048
H       -0.36012330      -0.82632057      -0.43645952
H       -0.36012330       0.83913943      -0.41126952    

Writing Mol8.xyz


In [37]:
%%file Mol9.xyz
3

S         -3.99240        2.55705        0.00000
O         -5.37459        2.17429        0.00000
O         -2.99297        1.84469        0.00000  

Writing Mol9.xyz


In [1]:
from ase.optimize import BFGS
from ase.calculators.mopac import MOPAC
from ase.io import read, write
from matplotlib import pyplot as plt
from ase.io.pov import get_bondpairs
import matplotlib.image as mpimg
import glob

#cmd = '/opt/mopac/MOPAC2016.exe PREFIX.mop 2>/dev/null'
cmd = path_mopac + 'MOPAC2016.exe PREFIX.mop 2>/dev/null'
names = glob.glob('*.xyz')            # lista com 'moleculas.xyz'
fig = plt.figure(figsize = (8,5)) 
for i, name in enumerate(names):
    mol = read(name)                         # importe a molécula
    natoms = len(mol)              # número de átomos da molécula
    mol.calc = MOPAC(command = cmd, method='PM6')    # calculador
    optimizer = BFGS(mol)                   # defina o otimizador
    optimizer.run()                         # otimize a geometria
    homo, lumo = mol.calc.get_homo_lumo_levels()    # HOMO e LUMO
    nom = mol.get_chemical_formula()                    # fórmula
    mol.set_positions(mol.get_positions() - mol.get_center_of_mass())
    bondpairs = get_bondpairs(mol)   # ligações químicas de 'mol'
    r = [0.1] * natoms       # raios dos átomos para renderização
    mydict = dict(canvas_height = 200, bondatoms = bondpairs)
    image = write('mol.pov',mol, radii=r, povray_settings=mydict)
    image.render()       # renderize a molécula, salve em mol.png
    fig.add_subplot(3,3,i+1)
    img = mpimg.imread('mol.png')             # importe 'mol.png'
    plt.imshow(img)                             # plote 'mol.png' 
    plt.title(f'{nom}\nLUMO: {lumo:.2f} eV\nHOMO: {homo:.2f} eV')
    plt.axis('off')                             # remova os eixos
    plt.tight_layout()
plt.savefig('ase2c.png', dpi = 400)        # Salve a figura final